In [1]:
import pandas as pd
import scattertext as st
import re, io
from pprint import pprint
import numpy as np
from scipy.stats import rankdata, hmean, norm
import spacy
from spacy.lemmatizer import Lemmatizer
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import gensim.corpora as corpora
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import os, pkgutil, json, urllib
from urllib.request import urlopen
from gensim.models import Word2Vec
from IPython.display import IFrame
from IPython.core.display import display, HTML
from scattertext import CorpusFromPandas, produce_scattertext_explorer
from gensim.models import Phrases
import codecs
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from sklearn.decomposition import PCA
from gensim.models.word2vec import LineSentence
import gensim
display(HTML("<style>.container { width:98% !important; }</style>"))

/Users/bassim/anaconda3/lib/python3.6/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [2]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])


In [25]:
data = pd.read_csv('reviews.csv')
data = data[['reviews','stars']]
data['sentiment'] = ['positive' if x in [4,5] else 'negative' for x in data['stars']]

In [4]:
data.head()

,reviews,stars,sentiment
0,Tel order to Wex for Atomos Shinobi. Little p...,1,negative
1,"Service was good fast delivery, the rwasi. For...",1,negative
2,"Since the take over of Calumet, Wex has failed...",1,negative
3,I returned a camera to them unused and they wo...,1,negative
4,The worst customer service I have ever experie...,1,negative


In [6]:
nlp = spacy.load('en')

In [60]:
# Terms that appear more frequently in the reviews corpus than any other English corpus

list(corpus.get_scaled_f_scores_vs_background().index[:10])

['wex',
 'parcelforce',
 'phoned',
 'knowledgable',
 'hesitation',
 'knowledgeable',
 'dslr',
 'packaged',
 'faultless',
 'arrived']

In [50]:
data['first_process'] = [gensim.utils.simple_preprocess(str(x), deacc=True) for x in data['reviews']] 

In [51]:
sentences = []
for sentence in data['first_process']:
    sample = []
    for word in sentence:
        if word not in stop_words:
            sample.append(wordnet_lemmatizer.lemmatize(word))
    sentences.append(sample)


In [13]:
bigram = gensim.models.Phrases([x for x in sentences], min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[sentences], threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

/Users/bassim/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [14]:
data_lemmatized = [trigram_mod[bigram_mod[sentences[x]]] for x in range(len(sentences))]

In [15]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1)]]


In [16]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [17]:
# Build LDA model. takes a little time
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=7, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=200,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [18]:
# Visualize the topics

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/Users/bassim/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.306244  0.128523       1        1  28.387987
3      0.166599 -0.324733       2        1  24.929070
0      0.133975  0.208044       3        1  17.749214
5     -0.093674 -0.195565       4        1   9.655528
4     -0.171562  0.067994       5        1   7.493230
6     -0.174104  0.060803       6        1   6.579361
2     -0.167478  0.054935       7        1   5.205610, topic_info=     Category         Freq          Term        Total  loglift  logprob
term                                                                   
38    Default  4100.000000           wex  4100.000000  30.0000  30.0000
15    Default  3589.000000       service  3589.000000  29.0000  29.0000
50    Default  2466.000000      delivery  2466.000000  28.0000  28.0000
113   Default  1583.000000           day  1583.000000  27.0000  27.0000
61    Default  1169.000000          lens  1169.000000  26.0000  26.0000
20    Default  1951.000000          good  1951.000000  25.0000  25.0000
84    Default  1316.000000         order  1316.000000  24.0000  24.0000
954   Default   832.000000  photographic   832.000000  23.0000  23.0000
13    Default  1629.000000         price  1629.000000  22.0000  22.0000
165   Default  1184.000000        camera  1184.000000  21.0000  21.0000
52    Default  1445.000000     excellent  1445.000000  20.0000  20.0000
194   Default  1725.000000        always  1725.000000  19.0000  19.0000
39    Default  1407.000000         great  1407.000000  18.0000  18.0000
42    Default  1004.000000       ordered  1004.000000  17.0000  17.0000
128   Default   967.000000       arrived   967.000000  16.0000  16.0000
160   Default  1196.000000          well  1196.000000  15.0000  15.0000
171   Default   828.000000          next   828.000000  14.0000  14.0000
37    Default   952.000000          used   952.000000  13.0000  13.0000
63    Default  1009.000000       product  1009.000000  12.0000  12.0000
71    Default   911.000000         would   911.000000  11.0000  11.0000
191   Default  1535.000000          time  1535.000000  10.0000  10.0000
137   Default  1213.000000          item  1213.000000   9.0000   9.0000
197   Default   762.000000       company   762.000000   8.0000   8.0000
310   Default   750.000000     recommend   750.000000   7.0000   7.0000
17    Default   799.000000          easy   799.000000   6.0000   6.0000
77    Default   748.000000       helpful   748.000000   5.0000   5.0000
454   Default   359.000000          need   359.000000   4.0000   4.0000
421   Default   394.000000         canon   394.000000   3.0000   3.0000
193   Default   340.000000        advice   340.000000   2.0000   2.0000
241   Default   633.000000         first   633.000000   1.0000   1.0000
...       ...          ...           ...          ...      ...      ...
839    Topic7   288.632202           kit   289.194458   2.9535  -3.1387
352    Topic7   132.159012   information   132.721756   2.9512  -3.9199
231    Topic7   121.407921         store   121.968391   2.9508  -4.0047
1074   Topic7   119.648491          look   120.209930   2.9508  -4.0193
269    Topic7   112.497360           see   113.060226   2.9504  -4.0810
921    Topic7   105.386169        superb   105.947052   2.9501  -4.1463
730    Topic7   101.488617        around   102.051132   2.9499  -4.1840
735    Topic7    99.831787        enough   100.393661   2.9498  -4.2004
812    Topic7    97.473221       support    98.035431   2.9497  -4.2243
842    Topic7    89.732826         since    90.297455   2.9492  -4.3071
1403   Topic7    88.118820  photographer    88.680321   2.9491  -4.3252
738    Topic7    77.371895         level    77.934090   2.9482  -4.4553
1049   Topic7    65.836632       part_ex    66.398354   2.9469  -4.6167
458    Topic7    64.514915        simply    65.079422   2.9467  -4.6370
271    Topic7    60.317875           set    60.879539   2.9462  -4.7043
1079   Topic7  